In [1]:
from math import floor, log2
from sympy import prime
import random

In [4]:
def gcd(a, b): # b > a
    if a>b:
        temp = a
        a = b
        b = temp
    q_0 = 0
    a_0 = a
    b_0 = b
    ### extended gcd ###
    s_0 = 1
    s_1 = 0
    t_0 = 0
    t_1 = 1
    ### extended gcd ###
    while True:
        q_0 = 0
        while b_0>=a_0: #  and a_0!=0
            b_0 -= a_0
            q_0+=1
        ### extended gcd ###
        temp_s = s_1
        s_1 = s_0-q_0*s_1
        s_0 = temp_s
        temp_t = t_1
        t_1 = t_0-q_0*t_1
        t_0 = temp_t
        # print(t_0, t_1, s_0, s_1)
        ### extended gcd ###
        if (q_0==0 or b_0==0): break
        rem = b_0
        b_0 = a_0
        a_0 = rem
        if (a_0==1):
            temp_s = s_1
            s_1 = s_0-q_0*s_1
            s_0 = temp_s
            temp_t = t_1
            t_1 = t_0-q_0*t_1
            t_0 = temp_t
            break
    return a_0, t_0, s_0
def lcm(a,b):
    return abs(a*b)//gcd(a,b)[0]
ascii32bit = "abcdefghijklmnopqrstuvwxyz.,!?' "
def ascii32bit_encode(s: str):
    acc = 0
    i = 0
    for c in s:
        acc+=(ascii32bit.index(c) if c in ascii32bit else 31)<<i
        i+=5
    return acc
# edge case where last char has leading zeros, need to adjust bit shift offset accordingly
def ascii32bit_decode(s: int):
    acc = s
    ans = ""
    n = round(log2(s)) - 5
    n = n+(5-n%5) if n%5!=0 else n
    for i in range(n, -1, -5):
        temp = acc >> i
        temp2 = temp << i
        ans=ascii32bit[temp]+ans
        acc -= temp2
    return ans
def fast_mod_exp(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m
# ported from https://stackoverflow.com/questions/2921406/calculate-primes-p-and-q-from-private-exponent-d-public-exponent-e-and-the
# given d e and n, find p and q
def pq_from_den(d, e, n):
    k = d*e-1
    if (k%2==0):
        r = k
        t = 0
        while (r%2==0):
            r = r >> 1
            t+=1
        success = False
        y = -1
        for i in range(1,101):
            g = -1
            g = random.randrange(0,n)
            y = fast_mod_exp(g,r,n)
            if y==1 or y==n-1:
                continue
            for j in range(1, t):
                x = fast_mod_exp(y,2,n)
                if x==1:
                    success = True
                    break
                if x==n-1:
                    continue
                y = x
            x = fast_mod_exp(y, 2, n)
            if (x==1):
                success=True
                break
        if (success):
            p = gcd(y-1, n)[0]
            q = n//p
            return p, q
        else:
            assert False, "not found"
    else:
        assert False, "not found"

In [22]:
c = 2757048617839190245303788170349245383273466387639798246893359430750735922748718434584285081419177420687542975400586145493620775809406535886702316058674001525290190527163250526070436240183674909832238790447423267718646298174986936873951720568059980762435342686228266433711365142474404806427627218891221309997379495373108110294245802987709230598
n = 42360895416872490494711845949439648321064353290438402862344728979674000306179352696402531952283449752524703928578237887411996665746582892622825061033969984886406584288402956282814393514178463185064912854429227656477325697945654499507848020346017892999014077250681328209894000463124025644052463027390506423213846398632352039953623295590590331641
e = 65537
d = 1292732209801257014959850037366362461542772885253777342946571523862062660975612331855365120535985771473357154846216271340219926629128061785642463372872422750092515198693957803464131513928878745901243964613248322519411193614161603354070159462472126981674903558316106266990982207398081256208018768860048718226767975300436456961826854924411869

In [21]:
from sympy.solvers import solve
from sympy import Symbol
from sympy.ntheory.continued_fraction import continued_fraction_convergents
from sympy import Rational
from sympy.ntheory import continued_fraction
# wiener's attack needs to become a cubic equation somehow
def attack(e,n):
    cf = continued_fraction(Rational(e,n))
    for c in list(continued_fraction_convergents(cf)):
        k = c.p
        d = c.q
        if (k==0 or d%2==0 or e*d%k!=1):
            continue
        phi = (e*d - 1)//k
        x = Symbol('x')
        print(d, solve(x**2 - ((n-phi)+1)*x + n, x))

In [23]:
attack(e,n)

1292732209801257014959850037366362461542772885253777342946571523862062660975612331855365120535985771473357154846216271340219926629128061785642463372872422750092515198693957803464131513928878745901243964613248322519411193614161603354070159462472126981674903558316106266990982207398081256208018768860048718226767975300436456961826854924411869 [1158 - sqrt(42360895416872490494711845949439648321064353290438402862344728979674000306179352696402531952283449752524703928578237887411996665746582892622825061033969984886406584288402956282814393514178463185064912854429227656477325697945654499507848020346017892999014077250681328209894000463124025644052463027390506423213846398632352039953623295590588990677)*I, 1158 + sqrt(423608954168724904947118459494396483210643532904384028623447289796740003061793526964025319522834497525247039285782378874119966657465828926228250610339699848864065842884029562828143935141784631850649128544292276564773256979456544995078480203460178929990140772506813282098940004631240256

In [15]:
42360895416872490494711845949439648321064353290438402862344728979674000306179352696402531952283449752524703928578237887411996665746582892622825061033969984886406584288402956282814393514178463185064912854429227656477325697945654499507848020346017892999014077250681328209894000463124025644052463027390506423213846398632352039953623295590590331641/1292732209801257014959850037366362461542772885253777342946571523862062660975612331855365120535985771473357154846216271340219926629128061785642463372872422750092515198693957803464131513928878745901243964613248322519411193614161603354070159462472126981674903558316106266990982207398081256208018768860048718226767975300436456961826854924411869 

32768.5

In [35]:
from sympy import lcm_list
from sympy import symbols
p,q,r = 17, 19, 29
n = p*q*r
e = 1003
ln = lcm(lcm(p-1, q-1), r-1)
d = 403

In [3]:
for i in range(1000000):
    try:
        pq_from_den(2, i, pow(2*3, 4))
        print("yay")
    except:
        pass

In [10]:
n = 95780963927892295077345800948872543933315542377531213521190180090405704795877442772536372715518117903283381686954178857387015371305045698819066879718212152259079548118842841732855275300958773347097251640155109877404063333288215124044686464595601587382908669192044161153727825663984136501238911020584290149701
e = 12071818393341018925780495229224517644829430398681780119398673841444544633015665613335221914798963240324102914906717081374135937973658810629804422333528117183086689655640380457205900346142057160309585204843427664727639678904528924234898565061649983656686141622789799911581959013672902682520443650802504823473

In [16]:
[c for c in list(continued_fraction_convergents(continued_fraction(Rational(e,n))))]

[0,
 1/646366104900628507479925018683181230771386442626888671473285761931031330487806165927682560267992885736678577423108135670109963314564030892821231686436211375046257599346978901732065756964439372950621982306624161259705596807080801677035079731236063490837451779158053133495491103699040628104009384430024359113383987650218228480913427462205934,
 1/646366104900628507479925018683181230771386442626888671473285761931031330487806165927682560267992885736678577423108135670109963314564030892821231686436211375046257599346978901732065756964439372950621982306624161259705596807080801677035079731236063490837451779158053133495491103699040628104009384430024359113383987650218228480913427462205935,
 2/12927322098012570149598500373663624615427728852537773429465715238620626609756123318553651205359857714733571548462162713402199266291280617856424633728724227500925151986939578034641315139288787459012439646132483225194111936141616033540701594624721269816749035583161062669909822073980812562080187688600487182

In [24]:
pq_from_den(d,e,n)

AssertionError: not found